<a href="https://colab.research.google.com/github/Pial-Khan/NER_BERT/blob/main/NER_BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

#import os
#print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.
from tqdm import tqdm, trange

In [3]:
input_data = pd.read_csv("clean_ner_all_campaign_data.csv", encoding="UTF-8")
#input_data

In [4]:
input_data = input_data.fillna(method="ffill")
input_data.tail(10)

,sentence_no,data,Word,position,Tag
165704,14716,এ ঘটনায় হামলাকারীদের দৃষ্টান্তমূলক শাস্তির দাব...,দৃষ্টান্তমূলক,4,O
165705,14716,এ ঘটনায় হামলাকারীদের দৃষ্টান্তমূলক শাস্তির দাব...,শাস্তির,5,O
165706,14716,এ ঘটনায় হামলাকারীদের দৃষ্টান্তমূলক শাস্তির দাব...,দাবিতে,6,O
165707,14716,এ ঘটনায় হামলাকারীদের দৃষ্টান্তমূলক শাস্তির দাব...,শিক্ষকদের,7,O
165708,14716,এ ঘটনায় হামলাকারীদের দৃষ্টান্তমূলক শাস্তির দাব...,কর্মবিরতির,8,O
165709,14716,এ ঘটনায় হামলাকারীদের দৃষ্টান্তমূলক শাস্তির দাব...,ঘোষণা,9,O
165710,14716,এ ঘটনায় হামলাকারীদের দৃষ্টান্তমূলক শাস্তির দাব...,দিয়েছে,10,O
165711,14716,এ ঘটনায় হামলাকারীদের দৃষ্টান্তমূলক শাস্তির দাব...,বিশ্ববিদ্যালয়,11,B-ORG
165712,14716,এ ঘটনায় হামলাকারীদের দৃষ্টান্তমূলক শাস্তির দাব...,শিক্ষক,12,I-ORG
165713,14716,এ ঘটনায় হামলাকারীদের দৃষ্টান্তমূলক শাস্তির দাব...,সমিতি,13,I-ORG


In [5]:
words_list = list(set(input_data["Word"].values))
words_list[:10]

['নারী',
 'সপ্তপর্ণা',
 'পড়ান',
 'প্রভাবশালীদের',
 'প্রবল',
 'বাঁচাব',
 'কোম্পানিজ',
 'বাণিজ্যমন্ত্রী',
 'বিব্রতকর',
 'মহাসড়কগুলো']

In [6]:
number_words = len(words_list); number_words # number of unique words in the corpus

20793

In [7]:
class RetrieveSentance(object):
    
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        function = lambda s: [(w, t) for w, t in zip(s["Word"].values.tolist(),
                                                           s["Tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence_no").apply(function)
        self.sentences = [s for s in self.grouped]
    
    def retrieve(self):
        try:
            s = self.grouped["Sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None

In [8]:
Sentences = RetrieveSentance(input_data)
Sentences

In [9]:
Sentences_list = [" ".join([s[0] for s in sent]) for sent in Sentences.sentences]
Sentences_list[0]

'এরশাদকে নিয়ে গণতন্ত্র হত্যা করেছে শেখ হাসিনা - মির্জা ফখরুল'

In [10]:
len(Sentences_list) #number of sentences 

11979

In [11]:
labels = [[s[1] for s in sent] for sent in Sentences.sentences]
print(labels[0])

['B-PER', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER']


In [12]:
labels [0] # list of lists of dimension (sentences,labels)

['B-PER', 'O', 'O', 'O', 'O', 'B-PER', 'I-PER', 'O', 'B-PER', 'I-PER']

In [13]:
tags2vals = list(set(input_data["Tag"].values))
tag2idx = {t: i for i, t in enumerate(tags2vals)}

In [14]:
tags2vals # 11 kinds of tags 

['I-ORG',
 'I-EVE',
 'B-Date',
 'I-LOC',
 'I-Date',
 'B-LOC',
 'B-PER',
 'B-ORG',
 'B-EVE',
 'O',
 'I-PER']

In [15]:
tag2idx # indexing the tag 

{'B-Date': 2,
 'B-EVE': 8,
 'B-LOC': 5,
 'B-ORG': 7,
 'B-PER': 6,
 'I-Date': 4,
 'I-EVE': 1,
 'I-LOC': 3,
 'I-ORG': 0,
 'I-PER': 10,
 'O': 9}

In [16]:
!pip install bert-tensorflow


     |████████████████████████████████| 71kB 6.4MB/s 


In [17]:
pip install pytorch_pretrained_bert

     |████████████████████████████████| 133kB 8.6MB/s 
     |████████████████████████████████| 102kB 3.6MB/s 
     |████████████████████████████████| 6.7MB 21.4MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
  Created wheel for boto3: filename=boto3-1.16.10-py2.py3-none-any.whl size=128451 sha256=22480d4571dd4afe270befe02b2e003e7c0ecc1a26dd06734e62e487d0917c50
  Stored in directory: /root/.cache/pip/wheels/36/dd/27/cf4486e326eeb099cbe273be73945d394f66914c0f14ac3b09
Successfully built boto3
ERROR: botocore 1.19.10 has requirement urllib3<1.26,>=1.25.4; python_version != "3.4", but you'll have urllib3 1.24.3 which is incompatible.


In [18]:
import torch
from torch.optim import Adam
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_pretrained_bert import BertTokenizer, BertConfig
from pytorch_pretrained_bert import BertForTokenClassification, BertAdam

In [19]:
max_seq_len = 75 # tokens
batch_s = 32 # batch size

In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
n_gpu = torch.cuda.device_count()

In [21]:
torch.cuda.get_device_name(0) 

'Tesla T4'

In [22]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased', do_lower_case=True)

The pre-trained model you are loading is a cased model but you have not set `do_lower_case` to False. We are setting `do_lower_case=False` for you but you may want to check this behavior.
100%|██████████| 213450/213450 [00:00<00:00, 19285973.78B/s]


In [23]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in Sentences_list]
print(tokenized_texts[0])

['এ', '##র', '##শ', '##া', '##দ', '##ক', '##ে', 'ন', '##ি', '##য়', '##ে', '[UNK]', 'হ', '##ত', '##্', '##য', '##া', 'ক', '##র', '##ে', '##ছ', '##ে', 'শ', '##ে', '##খ', 'হ', '##া', '##স', '##ি', '##ন', '##া', '-', 'ম', '##ি', '##র', '##্', '##জ', '##া', '[UNK]']


In [24]:
len(tokenized_texts)

11979

In [25]:
print(tokenized_texts[1])

['হ', '##ু', '##স', '##ে', '##ই', '##ন', 'ম', '##ু', '##হ', '##ম', '##্', '##ম', '##দ', 'এ', '##র', '##শ', '##া', '##দ', '##ক', '##ে', 'ন', '##ি', '##য়', '##ে', 'প', '##্', '##র', '##ধ', '##া', '##ন', '##ম', '##ন', '##্', '##ত', '##্', '##র', '##ী', 'শ', '##ে', '##খ', 'হ', '##া', '##স', '##ি', '##ন', '##া', 'দ', '##ে', '##শ', '##ে', '##র', '[UNK]', 'হ', '##ত', '##্', '##য', '##া', 'ক', '##র', '##ে', '##ছ', '##ে', 'ব', '##ল', '##ে', '[UNK]', 'ক', '##র', '##ে', '##ছ', '##ে', '##ন', 'ব', '##ি', '##এ', '##ন', '##প', '##ি', 'ম', '##হ', '##া', '##স', '##চ', '##ি', '##ব', 'ম', '##ি', '##র', '##্', '##জ', '##া', '[UNK]', 'ই', '##স', '##ল', '##া', '##ম', 'আ', '##ল', '##ম', '##গ', '##ী', '##র']


In [26]:
X = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=max_seq_len, dtype="long", truncating="post", padding="post")

Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (605 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (713 > 512). Running this sequence through BERT will result in indexing errors
Token indices sequence length is longer than the specified maximum  sequence length for this BERT model (517 > 512). Running this sequence through BERT will result in indexing errors


In [27]:
Y = pad_sequences([[tag2idx.get(l) for l in lab] for lab in labels],
                     maxlen=max_seq_len, value=tag2idx["O"], padding="post",
                     dtype="long", truncating="post")

In [28]:
X.shape # (sentences, maximum sequence length)

(11979, 75)

In [29]:
Y.shape

(11979, 75)

In [30]:
X

array([[  639, 28560, 28562, ...,     0,     0,     0],
       [  661, 28569, 28563, ...,   654, 28567, 28542],
       [  648, 28567, 28555, ..., 28548, 28567, 28561],
       ...,
       [  648, 28557, 28570, ...,     0,     0,     0],
       [  648, 28566, 28544, ...,     0,     0,     0],
       [  639,   100,   661, ...,     0,     0,     0]])

In [31]:
Y

array([[ 6,  9,  9, ...,  9,  9,  9],
       [ 6, 10, 10, ...,  9,  9,  9],
       [ 9,  9,  9, ...,  9,  9,  9],
       ...,
       [ 9,  9,  9, ...,  9,  9,  9],
       [ 9,  9,  9, ...,  9,  9,  9],
       [ 9,  9,  9, ...,  9,  9,  9]])

In [32]:
attention_masks = [[float(i>0) for i in ii] for ii in X]

In [33]:
len(attention_masks) # list of lists of shape (sentences, labels )

11979

In [34]:
attention_masks[0]

[1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [35]:
X_train, X_valid, Y_train, Y_valid = train_test_split(X, Y, 
                                                            random_state=20, test_size=0.1)
Mask_train, Mask_valid, _, _ = train_test_split(attention_masks, X,
                                             random_state=20, test_size=0.1)

In [36]:
X_train = torch.tensor(X_train)
X_valid = torch.tensor(X_valid)
Y_train = torch.tensor(Y_train)
Y_valid = torch.tensor(Y_valid)
Mask_train = torch.tensor(Mask_train)
Mask_valid = torch.tensor(Mask_valid)

In [37]:
data_train = TensorDataset(X_train, Mask_train, Y_train)
data_train_sampler = RandomSampler(data_train)
DL_train = DataLoader(data_train, sampler=data_train_sampler, batch_size=batch_s)

data_valid = TensorDataset(X_valid, Mask_valid, Y_valid)
data_valid_sampler = SequentialSampler(data_valid)
DL_valid = DataLoader(data_valid, sampler=data_valid_sampler, batch_size=batch_s)

In [38]:
model = BertForTokenClassification.from_pretrained("bert-base-cased", num_labels=len(tag2idx))

100%|██████████| 404400730/404400730 [00:09<00:00, 41776328.58B/s]


In [39]:
model.cuda();

In [40]:
FULL_FINETUNING = True
if FULL_FINETUNING:
    param_optimizer = list(model.named_parameters())
    no_decay = ['bias', 'gamma', 'beta']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.01},
        {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
         'weight_decay_rate': 0.0}
    ]
else:
    param_optimizer = list(model.classifier.named_parameters()) 
    optimizer_grouped_parameters = [{"params": [p for n, p in param_optimizer]}]
optimizer = Adam(optimizer_grouped_parameters, lr=3e-5)

In [42]:
pip install seqeval

     |████████████████████████████████| 51kB 5.3MB/s 
  Created wheel for seqeval: filename=seqeval-1.2.2-cp36-none-any.whl size=16171 sha256=dd1a9d2497bb52e388cd6f7aabb1e5a964ee0fbb65c6b4f2a341f80e077b9247
  Stored in directory: /root/.cache/pip/wheels/52/df/1b/45d75646c37428f7e626214704a0e35bd3cfc32eda37e59e5f
Successfully built seqeval


In [43]:
from seqeval.metrics import f1_score

def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=2).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [49]:
epochs = 5
max_grad_norm = 1.0

for _ in trange(epochs, desc="Epoch"):
    # TRAIN loop
    model.train()
    tr_loss = 0
    nb_tr_examples, nb_tr_steps = 0, 0
    for step, batch in enumerate(DL_train):
        # add batch to gpu
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        # forward pass
        loss = model(b_input_ids, token_type_ids=None,
                     attention_mask=b_input_mask, labels=b_labels)
        # backward pass
        loss.backward()
        # track train loss
        tr_loss += loss.item()
        nb_tr_examples += b_input_ids.size(0)
        nb_tr_steps += 1
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(parameters=model.parameters(), max_norm=max_grad_norm)
        # update parameters
        optimizer.step()
        model.zero_grad()
    # print train loss per epoch
    print("Train loss: {}".format(tr_loss/nb_tr_steps))
    # VALIDATION on validation set
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0
    predictions , true_labels = [], []
    for batch in DL_valid:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        with torch.no_grad():
            tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                                  attention_mask=b_input_mask, labels=b_labels)
            logits = model(b_input_ids, token_type_ids=None,
                           attention_mask=b_input_mask)
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
        true_labels.append(label_ids)
        
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        
        eval_loss += tmp_eval_loss.mean().item()
        eval_accuracy += tmp_eval_accuracy
        
        nb_eval_examples += b_input_ids.size(0)
        nb_eval_steps += 1
    eval_loss = eval_loss/nb_eval_steps
    print("Validation loss: {}".format(eval_loss))
    print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
    pred_tags = [tags2vals[p_i] for p in predictions for p_i in p]
    valid_tags = [tags2vals[l_ii] for l in true_labels for l_i in l for l_ii in l_i]
    #print("F1-Score: {}".format(f1_score(pred_tags, valid_tags)))
    




Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Train loss: 0.1143455079892092





Epoch:  20%|██        | 1/5 [03:05<12:21, 185.37s/it]

Validation loss: 0.1657932863423699
Validation Accuracy: 0.9670692355889725
Train loss: 0.10598194761649438





Epoch:  40%|████      | 2/5 [06:10<09:16, 185.40s/it]

...

Validation loss: 0.17341127932855957
Validation Accuracy: 0.968660714285714
Train loss: 0.09281937156745872





Epoch:  80%|████████  | 4/5 [12:21<03:05, 185.37s/it]

Validation loss: 0.16951388512787066
Validation Accuracy: 0.96859335839599
Train loss: 0.08713609312912476





Epoch: 100%|██████████| 5/5 [15:27<00:00, 185.41s/it]

Validation loss: 0.17496033542250333
Validation Accuracy: 0.9697916666666666


In [52]:
model.eval()
predictions = []
true_labels = []
eval_loss, eval_accuracy = 0, 0
nb_eval_steps, nb_eval_examples = 0, 0
for batch in DL_valid:
    batch = tuple(t.to(device) for t in batch)
    b_input_ids, b_input_mask, b_labels = batch

    with torch.no_grad():
        tmp_eval_loss = model(b_input_ids, token_type_ids=None,
                              attention_mask=b_input_mask, labels=b_labels)
        logits = model(b_input_ids, token_type_ids=None,
                       attention_mask=b_input_mask)
        
    logits = logits.detach().cpu().numpy()
    predictions.extend([list(p) for p in np.argmax(logits, axis=2)])
    label_ids = b_labels.to('cpu').numpy()
    true_labels.append(label_ids)
    tmp_eval_accuracy = flat_accuracy(logits, label_ids)

    eval_loss += tmp_eval_loss.mean().item()
    eval_accuracy += tmp_eval_accuracy

    nb_eval_examples += b_input_ids.size(0)
    nb_eval_steps += 1

pred_tags = [[tags2vals[p_i] for p_i in p] for p in predictions]
valid_tags = [[tags2vals[l_ii] for l_ii in l_i] for l in true_labels for l_i in l ]
print("Validation loss: {}".format(eval_loss/nb_eval_steps))
print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))
print("Validation F1-Score: {}".format(f1_score(pred_tags, valid_tags)))

Validation loss: 0.17496033542250333
Validation Accuracy: 0.9697916666666666
Validation F1-Score: 0.24284666177549522
